In [2]:
import pymongo
import jieba
from datetime import datetime, timedelta, date
from dateutil import parser

client = pymongo.MongoClient(host='127.0.0.1:27017')
collection_spider = client['test']['weibo']
collection_weibo = client['whut_final']['weibo']
collection_poster = client['test']['user']

In [8]:
used_words = set()
word_data = []
stop_words = ['##', '视频', '微博']

latest_weibo = collection_weibo.find_one({"user_id": int('1630856882')}, sort=[("created_at", -1)])
now = parser.parse(latest_weibo["created_at"])
date_ranges = {
    "最近五天": now - timedelta(days=5),
    "最近十四天": now - timedelta(days=14),
    "最近一个月": now - timedelta(days=30),
    "最近三个月": now - timedelta(days=90),
    "最近半年": now - timedelta(days=180),
    "全部": None
}

for label, start_date in date_ranges.items():
    if start_date:
        query = {"user_id": int('1630856882'), "created_at": {"$gte": start_date.isoformat()}}

    weibos = collection_weibo.find(query, {"text": 1, "_id": 0})
    word_frequency = {}

    for weibo in weibos:
        words = jieba.cut(weibo['text'])
        for word in words:
            if len(word) > 1 and word not in used_words and word not in stop_words:
                if word not in word_frequency:
                    word_frequency[word] = 1
                else:
                    word_frequency[word] += 1
    
    top_three = sorted(word_frequency.items(), key=lambda x: x[1], reverse=True)[:3]
    top_three_words = ' '.join([word[0] for word in top_three])

    used_words.update([word[0] for word in top_three])

    word_data.append({label, top_three_words})

In [9]:
word_data

[{'最近五天', '陶喆 SoulPowerII 世界'},
 {'大家 Soul Power', '最近十四天'},
 {'最近一个月', '音樂 見面 我們'},
 {'一起 可以 你們', '最近三个月'},
 {'希望 演出 2024', '最近半年'},
 {'全部', '時間 II 今天'}]

In [10]:
def LDA_Identify(f, k):
    return ('娱乐', 0.648592106)

In [14]:
lda = 1

In [15]:
LDA_Identify(lda, "#廖骏雄去世##廖骏雄离世# 据港媒，香港演员廖骏雄今晨离世，享年63岁。他曾在2021年4月不幸确诊胃癌第四期，化疗后表示身体状况有所转好。")

('娱乐', 0.648592106)